<a href="https://colab.research.google.com/github/2303a51219/aiml-2303a51219/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from typing import List, Dict, Set, Optional
from collections import defaultdict
import heapq

class User:
    def __init__(self, id: int, name: str, role: str):
        self.id = id
        self.name = name
        self.role = role

class Mentor:
    def __init__(self, id: int, name: str, skills: Set[str], industry: str, rating: float):
        self.id = id
        self.name = name
        self.skills = skills
        self.industry = industry
        self.rating = rating

class Booking:
    def __init__(self, mentor_id: int, mentee_id: int, start_time: str, end_time: str, status: str):
        self.mentor_id = mentor_id
        self.mentee_id = mentee_id
        self.start_time = start_time
        self.end_time = end_time
        self.status = status

class MentorConnect:
    def __init__(self):
        self.users: Dict[int, User] = {}
        self.mentors: Dict[int, Mentor] = {}
        self.bookings: Dict[int, List[Booking]] = defaultdict(list)
        self.connection_graph: Dict[int, List[int]] = defaultdict(list)

    def initialize_sample_data(self):
        # Create users
        self.users[101] = User(101, "Alice", "mentee")
        self.users[102] = User(102, "Bob", "mentee")

        # Create mentors
        self.mentors[1] = Mentor(1, "Dr. Smith", {"Java", "Algorithms"}, "Technology", 4.8)
        self.mentors[2] = Mentor(2, "Prof. Johnson", {"Python", "Machine Learning"}, "Technology", 4.5)
        self.mentors[3] = Mentor(3, "Ms. Davis", {"Marketing", "SEO"}, "Business", 4.9)

        # Create connections
        self.connection_graph[101].append(1)
        self.connection_graph[1].append(101)
        self.connection_graph[101].append(2)
        self.connection_graph[2].append(101)
        self.connection_graph[102].append(3)
        self.connection_graph[3].append(102)

        # Create existing bookings
        self.bookings[1].append(Booking(1, 100, "2023-12-15 10:00", "2023-12-15 11:00", "Confirmed"))

    def recommend_mentors(self, mentee_id: int, skill: str, industry: str, top_k: int) -> List[Mentor]:
        mentor_scores = []

        for mentor in self.mentors.values():
            score = self._calculate_match_score(mentor, skill, industry)
            # Using max-heap by pushing negative score
            heapq.heappush(mentor_scores, (-score, mentor))

        # Get top K mentors
        result = []
        for _ in range(min(top_k, len(mentor_scores))):
            if mentor_scores:
                result.append(heapq.heappop(mentor_scores)[1])
        return result

    def _calculate_match_score(self, mentor: Mentor, skill: str, industry: str) -> float:
        skill_match = 1.0 if skill in mentor.skills else 0.0
        industry_match = 1.0 if mentor.industry == industry else 0.0
        availability_score = 1.0 - (len(self.bookings.get(mentor.id, [])) / 10.0)
        return 0.4 * skill_match + 0.3 * industry_match + 0.2 * availability_score + 0.1 * mentor.rating

    def book_session(self, mentor_id: int, mentee_id: int, start_time: str, end_time: str) -> str:
        for booking in self.bookings.get(mentor_id, []):
            if self._is_time_overlap(booking.start_time, booking.end_time, start_time, end_time):
                return f"Failed: Time conflict with existing booking"

        new_booking = Booking(mentor_id, mentee_id, start_time, end_time, "Confirmed")
        self.bookings[mentor_id].append(new_booking)
        return f"Success: Booking confirmed with {self.mentors[mentor_id].name}"

    def _is_time_overlap(self, s1: str, e1: str, s2: str, e2: str) -> bool:
        # The original condition was incorrect, causing bookings to always be considered overlapping.
        # This line is corrected to check for overlaps correctly
        return not (e1 <= s2 or s1 >= e2)

def main():
    platform = MentorConnect()
    platform.initialize_sample_data()

    # Example usage
    mentee_id = 101
    desired_skill = "Java"
    target_industry = "Technology"
    top_mentors = 3

    # 1. Find recommended mentors
    recommended_mentors = platform.recommend_mentors(mentee_id, desired_skill, target_industry, top_mentors)

    print(f"Recommended Mentors for skill '{desired_skill}' in {target_industry}:")
    for mentor in recommended_mentors:
        print(f" - {mentor.name} (Rating: {mentor.rating}, Skills: {mentor.skills})")

    # 2. Book a session with the top mentor
    if recommended_mentors:
        selected_mentor = recommended_mentors[0]
        start_time = "2023-12-15 14:00"
        end_time = "2023-12-15 15:00"

        booking_result = platform.book_session(selected_mentor.id, mentee_id, start_time, end_time)
        print(f"\nBooking Status with {selected_mentor.name}: {booking_result}")

if __name__ == "__main__":
    main()

Recommended Mentors for skill 'Java' in Technology:
 - Dr. Smith (Rating: 4.8, Skills: {'Java', 'Algorithms'})
 - Prof. Johnson (Rating: 4.5, Skills: {'Python', 'Machine Learning'})
 - Ms. Davis (Rating: 4.9, Skills: {'SEO', 'Marketing'})

Booking Status with Dr. Smith: Success: Booking confirmed with Dr. Smith
